<center><img src="https://i.imgur.com/2rGzuN5.jpg" width="750px" style="border:1px solid black"></center>

# Introduction

In this project, I will show how one can finetune EfficientNet-B3 to detect Melanoma (a variety of skin cancer) from images. This problem is important because fast and accurate automated diagnosis can help reduce burden on doctors and let them focus on curing patients. I will use the **Efficientnet Pytorch** package to get the pretrained EfficientNet-B3 model and **PyTorch XLA** to train the model on TPU. At the end, we will run inference on the test set and test the model's predictions on some sample images.

# Preparing the ground

In this section, we will prepare the ground to train and test the model by installing packages, setting hyperparameters, and loading the data.

## Set up PyTorch XLA

* We need to install a package called **PyTorch XLA** to take advantage of TPUs with PyTorch.
* Training on TPUs can be significantly faster than on CPUs or GPUs. So we will use TPUs for this project.

In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py

In [ ]:
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

## Install additional packages

* We will now install two other packages: **colored** and **efficientnet_pytorch**.
* **colored** helps add foreground colors to text output in Python to make training logs more readable.
* **efficientnet_pytorch** will help us load the EfficientNet-B3 architecture with the pretrained weights.

In [ ]:
!export XLA_USE_BF16=1
!pip install -q colored
!pip install -q efficientnet_pytorch

## Import necessary libraries

* Now, we import all the libraries we need.
* **colored, matplotlib, tqdm, and plotly** for visualization.
* **numpy, pandas, torch, torchvision, albumentations, and efficientnet_pytorch** for modelling.

In [ ]:
import os
import gc
import cv2
import time
import numpy as np
import pandas as pd

from colored import fg, attr
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torch import FloatTensor, LongTensor, DoubleTensor
from torch.utils.data.sampler import WeightedRandomSampler

import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

from efficientnet_pytorch import EfficientNet
from albumentations import Normalize, VerticalFlip, HorizontalFlip, Compose

## Define key hyperparameters and paths

* Here we define the key hyperparameters: image height and width, train/val split, batch size, epochs, and LR.
* These hyperparameters can be tuned for performance, but runtime will change if you change **EPOCHS, H, or W.**

### Unzip ZIP files

In [ ]:
!unzip ../input/siim-isic-melanoma-resized-images/test.zip
!unzip ../input/siim-isic-melanoma-resized-images/train_1.zip
!unzip ../input/siim-isic-melanoma-resized-images/train_2.zip

### Define hyperparameters and paths

In [ ]:
W = 512
H = 512
B = 0.5
SPLIT = 0.8
SAMPLE = True
MU = [0.485, 0.456, 0.406]
SIGMA = [0.229, 0.224, 0.225]

EPOCHS = 5
LR = 1e-3, 1e-3
BATCH_SIZE = 32
VAL_BATCH_SIZE = 32
MODEL = 'efficientnet-b3'
IMG_PATHS = ['../working/test',
             '../working/train_1',
             '../working/train_2']

### Get image path dictionary

In [ ]:
PATH_DICT = {}
for folder_path in tqdm(IMG_PATHS):
    for img_path in os.listdir(folder_path):
        PATH_DICT[img_path] = folder_path + '/'

## Set random seeds

* The next step is to set the random seed for NumPy and PyTorch.
* Setting the random seed helps us keep training determinstic and ensure reproducible results.

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

## Load the training and testing metadata

* Now we need to load the training and testing metadata.
* These dataframes contain the image file names and targets which we need for training.

In [ ]:
print(os.listdir('../input/siim-isic-melanoma-classification'))

In [ ]:
test_df = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
train_df = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')

In [ ]:
test_df.head(10)

In [ ]:
train_df.head(10)

# EDA

Now, I will do some basic visualization to gain some insights from the data.

## Values in categorical columns

* First I will look at the set of values present in each categorical column.
* The categorical columns include **target, sex, anatom_site_general_challenge, diagnosis, and benign_malignant**.

In [ ]:
sfn = lambda x: "🔎 " + str(x)
fs = (fg('#7efc79'), attr('reset'))
gs = (fg('#fac3c3'), attr('reset'))

for column in train_df.columns[[7, 2, 6, 4, 5]]:
    column_string = ("%s" + column + "%s 🎯\n") % gs
    column_values = set(train_df[column].apply(sfn))
    column_values = "\n".join(sorted(column_values, key=len))
    hyphens = ("%s" + ''.join(['-']*len(column)) + "\n%s") % gs
    print(hyphens + column_string + hyphens + "\n" + ("%s" + column_values + "\n%s") % fs)

## Target vs. Proportion (per sex)

Let us understand how **sex** affects the **target**.

In [ ]:
x = ['healthy', 'melanoma']
x_1 = len(train_df.query('sex == "male" and target == 0'))
x_2 = len(train_df.query('sex == "male" and target == 1'))
x_3 = len(train_df.query('sex == "female" and target == 0'))
x_4 = len(train_df.query('sex == "female" and target == 1'))

x_1, x_3 = x_1/sum([x_1, x_3]), x_3/sum([x_1, x_3])
x_2, x_4 = x_2/sum([x_2, x_4]), x_4/sum([x_2, x_4])
fig = go.Figure(data=[go.Bar(name='male', y=x, x=[x_1, x_2], orientation="h",
                             marker=dict(color='#02cc45', line=dict(width=0.5, color='black'))),
                      go.Bar(name='female', y=x, x=[x_3, x_4], orientation="h",
                             marker=dict(color='#f7766f', line=dict(width=0.5, color='black')))])

fig.update_layout(xaxis_title='Target', yaxis_title='Proportion', template="plotly_white",
                  title='Target vs. Proportion (per sex)', barmode='stack', paper_bgcolor="#edebeb"); fig.show()

From the graph above, we can see that the proportion of males (in green) under "melanoma" is much higher than under "healthy". This might suggest that most patients suffering from melanoma in this dataset are males.

## Target vs. Proportion (per benign_malignant)

Now let us understand how **benign_malignant** affects the **target**.

In [ ]:
x = ['healthy', 'melanoma']
x_1 = len(train_df.query('benign_malignant == "benign" and target == 0'))
x_2 = len(train_df.query('benign_malignant == "benign" and target == 1'))
x_3 = len(train_df.query('benign_malignant == "malignant" and target == 0'))
x_4 = len(train_df.query('benign_malignant == "malignant" and target == 1'))

x_1, x_3 = x_1/sum([x_1, x_3]), x_3/sum([x_1, x_3])
x_2, x_4 = x_2/sum([x_2, x_4]), x_4/sum([x_2, x_4])
fig = go.Figure(data=[go.Bar(name='benign', y=x, x=[x_1, x_2], orientation="h",
                             marker=dict(color='#02cc45', line=dict(width=0.5, color='black'))),
                      go.Bar(name='malignant', y=x, x=[x_3, x_4], orientation="h",
                             marker=dict(color='#f7766f', line=dict(width=0.5, color='black')))])

fig.update_layout(xaxis_title='Target', yaxis_title='Proportion', template="plotly_white",
                  title='Target vs. Proportion (per benign_malignant)', barmode='stack', paper_bgcolor="#edebeb"); fig.show()

We can clearly see from the bar chart above that the **benign_malignant** feature perfectly mirrors the target.

## Target vs. Proportion (per anatom_site_general_challenge)

Next, let us look at how **anatom_site_general_challenge** affects the **target**.

In [ ]:
x = ['healthy', 'melanoma']

x_2 = len(train_df.query('anatom_site_general_challenge == "torso" and target == 0'))
x_3 = len(train_df.query('anatom_site_general_challenge == "head/neck" and target == 0'))
x_4 = len(train_df.query('anatom_site_general_challenge == "palms/soles" and target == 0'))
x_5 = len(train_df.query('anatom_site_general_challenge == "oral/genital" and target == 0'))
x_6 = len(train_df.query('anatom_site_general_challenge == "upper extremity" and target == 0'))
x_7 = len(train_df.query('anatom_site_general_challenge == "lower extremity" and target == 0'))
x_1 = len(train_df.query('anatom_site_general_challenge != anatom_site_general_challenge and target == 0'))

x_9 = len(train_df.query('anatom_site_general_challenge == "torso" and target == 1'))
x_10 = len(train_df.query('anatom_site_general_challenge == "head/neck" and target == 1'))
x_11 = len(train_df.query('anatom_site_general_challenge == "palms/soles" and target == 1'))
x_12 = len(train_df.query('anatom_site_general_challenge == "oral/genital" and target == 1'))
x_13 = len(train_df.query('anatom_site_general_challenge == "upper extremity" and target == 1'))
x_14 = len(train_df.query('anatom_site_general_challenge == "lower extremity" and target == 1'))
x_8 = len(train_df.query('anatom_site_general_challenge != anatom_site_general_challenge and target == 1'))

total = sum([x_1, x_2, x_3, x_4, x_5, x_6, x_7])
x_1, x_2, x_3, x_4, x_5, x_6, x_7 = [y/total for y in [x_1, x_2, x_3, x_4, x_5, x_6, x_7]]

total = sum([x_8, x_9, x_10, x_11, x_12, x_13, x_14])
x_8, x_9, x_10, x_11, x_12, x_13, x_14 = [y/total for y in [x_8, x_9, x_10, x_11, x_12, x_13, x_14]]


fig = go.Figure(data=[go.Bar(name='nan', y=x, x=[x_1, x_8], orientation="h",
                             marker=dict(color='red', line=dict(width=0.5, color='black'))),
                      go.Bar(name='torso', y=x, x=[x_2, x_9], orientation="h",
                             marker=dict(color='orange', line=dict(width=0.5, color='black'))),
                      go.Bar(name='head/neck', y=x, x=[x_3, x_10], orientation="h",
                             marker=dict(color='gold', line=dict(width=0.5, color='black'))),
                      go.Bar(name='palms/soles', y=x, x=[x_4, x_11], orientation="h",
                             marker=dict(color='green', line=dict(width=0.5, color='black'))),
                      go.Bar(name='oral/genital', y=x, x=[x_5, x_12], orientation="h",
                             marker=dict(color='blue', line=dict(width=0.5, color='black'))),
                      go.Bar(name='upper extremity', y=x, x=[x_6, x_13], orientation="h",
                             marker=dict(color='violet', line=dict(width=0.5, color='black'))),
                      go.Bar(name='lower extremity', y=x, x=[x_7, x_14], orientation="h",
                             marker=dict(color='indigo', line=dict(width=0.5, color='black')))])

fig.update_layout(xaxis_title='Target', yaxis_title='Proportion', template="plotly_white",
                  title='Target vs. Proportion (per anatom_site_general_challenge)', barmode='stack', paper_bgcolor="#edebeb"); fig.show()

From the bar charts above, we can see two major differences between melanoma images and non-melanoma images. The melanoma images tend to be more from the "head/neck" and "upper extremity" regions (yellow and pink) as compared to non-melanoma images. On the other hand, non-melanoma images tend to be more from the "torso" and "lower extremity" regions (orange and purple) as compared to melanoma images. This might suggest that **melanoma images are more likely to be from lower regions of the body than upper regions.**

## Target vs. Proportion (per diagnosis)

Now let us look at how the **diagnosis** affects the **target**.

In [ ]:
x = ['healthy', 'melanoma']

x_1 = len(train_df.query('diagnosis == "nevus" and target == 0'))
x_2 = len(train_df.query('diagnosis == "unknown" and target == 0'))
x_3 = len(train_df.query('diagnosis == "melanoma" and target == 0'))
x_4 = len(train_df.query('diagnosis == "lentigo NOS" and target == 0'))
x_5 = len(train_df.query('diagnosis == "solar lentigo" and target == 0'))
x_6 = len(train_df.query('diagnosis == "lichenoid keratosis" and target == 0'))
x_7 = len(train_df.query('diagnosis == "cafe-au-lait macule" and target == 0'))
x_8 = len(train_df.query('diagnosis == "seborrheic keratosis" and target == 0'))
x_9 = len(train_df.query('diagnosis == "atypical melanocytic proliferation" and target == 0'))

x_10 = len(train_df.query('diagnosis == "nevus" and target == 1'))
x_11 = len(train_df.query('diagnosis == "unknown" and target == 1'))
x_12 = len(train_df.query('diagnosis == "melanoma" and target == 1'))
x_13 = len(train_df.query('diagnosis == "lentigo NOS" and target == 1'))
x_14 = len(train_df.query('diagnosis == "solar lentigo" and target == 1'))
x_15 = len(train_df.query('diagnosis == "lichenoid keratosis" and target == 1'))
x_16 = len(train_df.query('diagnosis == "cafe-au-lait macule" and target == 1'))
x_17 = len(train_df.query('diagnosis == "seborrheic keratosis" and target == 1'))
x_18 = len(train_df.query('diagnosis == "atypical melanocytic proliferation" and target == 1'))

total = sum([x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9])
x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9 = [y/total for y in [x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9]]

total = sum([x_10, x_11, x_12, x_13, x_14, x_15, x_16, x_17, x_18])
x_10, x_11, x_12, x_13, x_14, x_15, x_16, x_17, x_18 = [y/total for y in [x_10, x_11, x_12, x_13, x_14, x_15, x_16, x_17, x_18]]


fig = go.Figure(data=[go.Bar(name='nevus', y=x, x=[x_1, x_10], orientation="h",
                             marker=dict(color='red', line=dict(width=0.5, color='black'))),
                      go.Bar(name='unknown', y=x, x=[x_2, x_11], orientation="h",
                             marker=dict(color='orange', line=dict(width=0.5, color='black'))),
                      go.Bar(name='melanoma', y=x, x=[x_3, x_12], orientation="h",
                             marker=dict(color='gold', line=dict(width=0.5, color='black'))),
                      go.Bar(name='lentigo NOS', y=x, x=[x_4, x_13], orientation="h",
                             marker=dict(color='green', line=dict(width=0.5, color='black'))),
                      go.Bar(name='solar lentigo', y=x, x=[x_5, x_14], orientation="h",
                             marker=dict(color='blue', line=dict(width=0.5, color='black'))),
                      go.Bar(name='lichenoid keratosis', y=x, x=[x_6, x_15], orientation="h",
                             marker=dict(color='violet', line=dict(width=0.5, color='black'))),
                      go.Bar(name='cafe-au-lait macule', y=x, x=[x_7, x_16], orientation="h",
                             marker=dict(color='indigo', line=dict(width=0.5, color='black'))),
                      go.Bar(name='seborrheic keratosis', y=x, x=[x_8, x_17], orientation="h",
                             marker=dict(color='goldenrod', line=dict(width=0.5, color='black'))),
                      go.Bar(name='atypical melanocytic proliferation', y=x, x=[x_9, x_18], orientation="h",
                             marker=dict(color='silver', line=dict(width=0.5, color='black')))])

fig.update_layout(xaxis_title='Target', yaxis_title='Proportion', template="plotly_white",
                  title='Target vs. Proportion (per anatom_site_general_challenge)', barmode='stack', paper_bgcolor="#edebeb"); fig.show()

The above graph clearly shows that all melanoma images are diagnosed as melanoma (as expected), and **most non-melanoma images are diagnosed as either "unknown" or "nevus".**

## Approximate age vs. Target

Finally let us see how the **approximate age** affects the **target**.

In [ ]:
nums_1 = train_df.query("target == 1")["age_approx"]
nums_2 = train_df.query("target == 0")["age_approx"]

nums_1 = nums_1.fillna(nums_1.mean())
nums_2 = nums_2.fillna(nums_2.mean())

fig = ff.create_distplot(hist_data=[nums_1, nums_2],
                         group_labels=["0", "1"],
                         colors=["darkorange", "dodgerblue"], show_hist=False)

fig.update_layout(title_text="Approximate age vs. Target", xaxis_title="Approximate age",
                  template="plotly_white", paper_bgcolor="#edebeb")
fig.show()

We can see from the distribution plot above that images with melanoma tend to come from younger patients with an average age of around 50, while non-melanoma images tend to come from older patients with an average age of around 65.

## Display sample images

* Now, I will show few sample images from the test directory to get an idea of what these skin images look like.

In [ ]:
def display_images(num):
    sq_num = np.sqrt(num)
    assert sq_num == int(sq_num)

    sq_num = int(sq_num)
    image_ids = os.listdir(IMG_PATHS[0])
    fig, ax = plt.subplots(nrows=sq_num, ncols=sq_num, figsize=(20, 20))

    for i in range(sq_num):
        for j in range(sq_num):
            idx = i*sq_num + j
            ax[i, j].axis('off')
            img = cv2.imread(IMG_PATHS[0] + '/' + image_ids[idx])
            ax[i, j].imshow(img); ax[i, j].set_title('Test Image {}'.format(idx), fontsize=12)

    plt.show()

In [ ]:
display_images(36)

<center><img src="https://i.imgur.com/ZwkZIbN.jpg" width="600px"></center>

# Modeling

* Now since we understand the dataset better, it is time to build a machine learning pipeline to solve this problem.
* I will use a classic image classification pipeline with image augmentation and a pretrained EfficientNet model.

## Define PyTorch dataset

* The first step is to build a PyTorch dataset to generate data for our model.
* A PyTorch dataset has three fundamental functions: <code>init</code>, <code>len</code>, and <code>getitem</code>.
* The <code>init</code> function initializes all the components required for data loading (image path, transforms, etc).
* The <code>len</code> function simply returns the length of the dataset. This indicates the number of retrievable samples.
* The <code>getitem</code> function returns a data point at a given index <code>idx</code>. The actual logic is written in this function.
* The <code>getitem</code> function does 3 things: gets the target, loads the image, resizes it to (512, 512), and randomly flips.

In [ ]:
def to_tensor(data):
    return [FloatTensor(point) for point in data]

def set_image_transformations(dataset, aug):
    norm = Normalize(mean=MU, std=SIGMA, p=1)
    vflip, hflip = VerticalFlip(p=0.5), HorizontalFlip(p=0.5)
    dataset.transformation = Compose([norm, vflip, hflip]) if aug else norm

class SIIMDataset(Dataset):
    def __init__(self, df, aug, targ, ids):
        set_image_transformations(self, aug)
        self.df, self.targ, self.aug, self.image_ids = df, targ, aug, ids

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, i):
        image_id = self.image_ids[i]
        target = [self.df.target[i]] if self.targ else 0
        image = cv2.imread(PATH_DICT[image_id] + image_id)
        return to_tensor([self.transformation(image=image)['image'], target])

## Define EfficientNet-B3 model

* Now we define the actual model which we are going to train.
* The <code>CancerNet</code> model uses an **EfficientNet-B3** backbone with a **(Pooling + Dense)** head for classification.

In [ ]:
def GlobalAveragePooling(x):
    return x.mean(axis=-1).mean(axis=-1)

class CancerNet(nn.Module):
    def __init__(self, features):
        super(CancerNet, self).__init__()
        self.avgpool = GlobalAveragePooling
        self.dense_output = nn.Linear(features, 1)
        self.efn = EfficientNet.from_pretrained(MODEL)
        
    def forward(self, x):
        x = x.view(-1, 3, H, W)
        x = self.efn.extract_features(x)
        return self.dense_output(self.avgpool(x))

## Define binary cross entropy and accuracy

* Next, we define our loss function (**binary cross entropy**) and evaluation metric (**accuracy**).

In [ ]:
def bce(y_true, y_pred):
    return nn.BCEWithLogitsLoss()(y_pred, y_true)

def acc(y_true, y_pred):
    y_true = y_true.squeeze()
    y_pred = nn.Sigmoid()(y_pred).squeeze()
    return (y_true == torch.round(y_pred)).float().sum()/len(y_true)

## Define helper function for training logs

* Now we define a simple helper function to produce training logs (with foreground colors).

In [ ]:
def print_metric(data, batch, epoch, start, end, metric, typ):
    t = typ, metric, "%s", data, "%s"
    if typ == "Train": pre = "BATCH %s" + str(batch-1) + "%s  "
    if typ == "Val": pre = "\nEPOCH %s" + str(epoch+1) + "%s  "
    time = np.round(end - start, 1); time = "Time: %s{}%s s".format(time)
    fonts = [(fg(211), attr('reset')), (fg(212), attr('reset')), (fg(213), attr('reset'))]
    print(pre % fonts[0] + "{} {}: {}{}{}".format(*t) % fonts[1] + "  " + time % fonts[2])

## Split train/val (80/20)

* We now split the data into training and validation sets (train: 80%, val: 20%).

In [ ]:
split = int(SPLIT*len(train_df))
train_df, val_df = train_df.loc[:split], train_df.loc[split:]
train_df, val_df = train_df.reset_index(drop=True), val_df.reset_index(drop=True)

## Define PyTorch dataloaders, model, and optimizer

* Next we define the PyTorch dataloader (for loading the train and val data).
* We also define our model (<code>CancerNet</code>) and optimizer (<code>ADAM</code>) for training.
* We use <code>WeightedRandomSampler</code> to fix data imbalance (with weight <code>B</code> for 1s)

### Define sampling weights

In [ ]:
C = np.array([B, (1 - B)])*2
ones = len(train_df.query('target == 1'))
zeros = len(train_df.query('target == 0'))

weightage_fn = {0: C[1]/zeros, 1: C[0]/ones}
weights = [weightage_fn[target] for target in train_df.target]

### Define PyTorch datasets

In [ ]:
length = len(train_df)
val_ids = val_df.image_name.apply(lambda x: x + '.jpg')
train_ids = train_df.image_name.apply(lambda x: x + '.jpg')

val_set = SIIMDataset(val_df, False, True, ids=val_ids)
train_set = SIIMDataset(train_df, True, True, ids=train_ids)

### Define sampling procedure and DataLoader

In [ ]:
train_sampler = WeightedRandomSampler(weights, length)
if_sample, if_shuffle = (train_sampler, False), (None, True)
sample_fn = lambda is_sample, sampler: if_sample if is_sample else if_shuffle

sampler, shuffler = sample_fn(SAMPLE, train_sampler)
val_loader = DataLoader(val_set, VAL_BATCH_SIZE, shuffle=False)
train_loader = DataLoader(train_set, BATCH_SIZE, sampler=sampler, shuffle=shuffler)

### Define model and optimizer

In [ ]:
device = xm.xla_device()
network = CancerNet(features=1536).to(device)
optimizer = Adam([{'params': network.efn.parameters(), 'lr': LR[0]},
                  {'params': network.dense_output.parameters(), 'lr': LR[1]}])

## Train the model on one TPU core

* Now, we train the model on one TPU core.
* First, we do a **training loop**, where we train the model with back-prop to adjust the parameters.
* Second, we do a **validation loop**, to check the model's performance on unseen data after each epoch.
* The training loop uses **forward-prop and back-prop**, while the validation loop uses **only forward-prop.**
* We use the <code>torch.no_grad()</code> flag for the validation loop as no gradients are needed for forward-prop.

In [ ]:
print("STARTING TRAINING ...\n")

start = time.time()
train_batches = len(train_loader) - 1

for epoch in range(EPOCHS):
    fonts = (fg(48), attr('reset'))
    print(("EPOCH %s" + str(epoch+1) + "%s") % fonts)
    
    batch = 1
    network.train()
    for train_batch in train_loader:
        train_img, train_targ = train_batch
        train_targ = train_targ.view(-1, 1)
        train_img, train_targ = train_img.to(device), train_targ.to(device)
        
        if batch >= train_batches: break
        train_preds = network.forward(train_img)
        train_acc = acc(train_targ, train_preds)
        train_loss = bce(train_targ, train_preds)
            
        optimizer.zero_grad()
        train_loss.backward()
        xm.optimizer_step(optimizer, barrier=True)
            
        end = time.time()
        batch = batch + 1
        log = batch % 10 == 1
        accuracy = np.round(train_acc.item(), 3)
        if log: print_metric(accuracy, batch, 0, start, end, "acc", "Train")
            
    network.eval()
    val_loss, val_acc, val_points = 0, 0, 0
        
    with torch.no_grad():
        for val_batch in tqdm(val_loader):
            val_img, val_targ = val_batch
            val_targ = val_targ.view(-1, 1)
            val_img, val_targ = val_img.to(device), val_targ.to(device)

            val_points += len(val_targ)
            val_preds = network.forward(val_img)
            val_acc += acc(val_targ, val_preds).item()*len(val_preds)
            val_loss += bce(val_targ, val_preds).item()*len(val_preds)
        
    end = time.time()
    val_acc /= val_points
    val_loss /= val_points
    accuracy = np.round(val_acc, 3)
    print_metric(accuracy, 0, epoch, start, end, "acc", "Val")
    
    print("")

print("ENDING TRAINING ...")

## Run inference on the test data

* Next I will run inference on the test data and store the test predictions in a list.
* These predictions are logits and will be converted to probabilities later using <code>sigmoid</code>.

In [ ]:
test_ids = test_df.image_name.apply(lambda x: x + '.jpg')

In [ ]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

test_set = SIIMDataset(test_df, False, False, test_ids)
test_loader = tqdm(DataLoader(test_set, VAL_BATCH_SIZE, shuffle=False))

network.eval()
test_preds = []
with torch.no_grad():
    for test_batch in test_loader:
        test_img, _ = test_batch
        test_img = test_img.to(device)
        test_preds.extend(network.forward(test_img).squeeze().detach().cpu().numpy())

## Visualize sample test predictions

* Now since the model is trained, we will visualize predictions made on unseen test images.

In [ ]:
def display_preds(num):
    sq_num = np.sqrt(num)
    assert sq_num == int(sq_num)

    sq_num = int(sq_num)
    image_ids = os.listdir(IMG_PATHS[0])
    few_preds = sigmoid(np.array(test_preds[:num]))
    pred_dict = {0: '"No Melanoma"', 1: '"Melanoma"'}
    fig, ax = plt.subplots(nrows=sq_num, ncols=sq_num, figsize=(20, 20))
    norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], p=1)

    for i in range(sq_num):
        for j in range(sq_num):
            idx = i*sq_num + j
            ax[i, j].axis('off')
            pred = few_preds[idx]
            img = cv2.imread(IMG_PATHS[0] + '/' + image_ids[idx])
            ax[i, j].imshow(img); ax[i, j].set_title('Prediction: {}'.format(pred_dict[round(pred.item())]), fontsize=12)

    plt.show()

In [ ]:
display_preds(16)

## Sigmoid the logits and generate submission file

* Now we apply the sigmoid function to all the logits and prepare the final submission file.

In [ ]:
path = '../input/siim-isic-melanoma-classification/'
sample_submission = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
sample_submission.target = sigmoid(np.array(test_preds))

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission.csv', index=False)

# Takeaways

* Training is very fast on TPUs and should be used when possible.
* Pretrained models like EfficientNet-B3 can generalize effectively to new tasks.